In [16]:
import pandas as pd
import requests
import time
import geopandas as gpd
from dateutil.relativedelta import relativedelta
import datetime as dt
import math

In [17]:
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", 10)

In [18]:
# Read in new file
file_loc_raw = "../datasets/resale_hdb_price_raw_25jun.csv"
df_raw = pd.read_csv(file_loc_raw, parse_dates=["month", "lease_commence_date"])

df_raw["floor_area_sqft"] = df_raw["floor_area_sqm"] * 10.7639
df_raw["price_per_sqft"] = (
    df_raw["resale_price"] / df_raw["floor_area_sqft"]
)

df_raw = df_raw.drop(["remaining_lease", "lease_commence_date"], axis=1)

In [19]:
df_raw.head(1)

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,resale_price,floor_area_sqft,price_per_sqft
0,2017-01-01,ANG MO KIO,2 ROOM,406,ANG MO KIO AVE 10,10 TO 12,44.0,Improved,232000.0,473.6116,489.852867


In [20]:
df_raw.shape

(182483, 11)

### Begin the synchronisation process

#### 1. Open the masterlist of HDB addresses. This will be used to perform a left join to the new data

In [21]:
masterlist_file_loc = "../datasets/hdb_resale_flat_address_masterlist.csv"
# masterlist_file_loc = "./temp_masterlist.csv"
masterlist_df = pd.read_csv(
    masterlist_file_loc, index_col=0, dtype={"postal": "object"}, parse_dates=["lease_commence_date"]
)

masterlist_df

,town,block,street_name,blk_no,road_name,building,postal,address,lease_commence_date,planning_area_ura,region_ura,x,y,latitude,longitude,closest_mrt_station,distance_to_mrt_meters,transport_type,line_color,distance_to_cbd,closest_pri_school,distance_to_pri_school_meters
0,ANG MO KIO,205,ANG MO KIO AVE 1,205,ANG MO KIO AVENUE 1,NIL,560205,205 ANG MO KIO AVENUE 1 SINGAPORE 560205,1977-01-01,ANG MO KIO,NORTH-EAST REGION,29142.244275,38774.891527,1.366941,103.843582,Ang Mo Kio,742.768808,MRT,Red,9199.172507,ANG MO KIO PRIMARY SCHOOL,512.545254
1,ANG MO KIO,207,ANG MO KIO AVE 1,207,ANG MO KIO AVENUE 1,ANG MO KIO 22,560207,207 ANG MO KIO AVENUE 1 ANG MO KIO 22 SINGAPOR...,1976-07-01,ANG MO KIO,NORTH-EAST REGION,29060.485578,38651.052977,1.365821,103.842848,Ang Mo Kio,874.305666,MRT,Red,9084.169820,ANG MO KIO PRIMARY SCHOOL,527.275749
2,ANG MO KIO,208,ANG MO KIO AVE 1,208,ANG MO KIO AVENUE 1,ANG MO KIO 22,560208,208 ANG MO KIO AVENUE 1 ANG MO KIO 22 SINGAPOR...,1976-07-01,ANG MO KIO,NORTH-EAST REGION,29045.715075,38609.483079,1.365445,103.842715,Ang Mo Kio,908.966103,MRT,Red,9044.410980,ANG MO KIO PRIMARY SCHOOL,549.157654
3,ANG MO KIO,215,ANG MO KIO AVE 1,215,ANG MO KIO AVENUE 1,ANG MO KIO 22,560215,215 ANG MO KIO AVENUE 1 ANG MO KIO 22 SINGAPOR...,1976-04-01,ANG MO KIO,NORTH-EAST REGION,28924.303291,38732.591142,1.366558,103.841624,Mayflower,781.530965,MRT,Brown,9180.500545,ANG MO KIO PRIMARY SCHOOL,377.594832
4,ANG MO KIO,216,ANG MO KIO AVE 1,216,ANG MO KIO AVENUE 1,ANG MO KIO 22,560216,216 ANG MO KIO AVENUE 1 ANG MO KIO 22 SINGAPOR...,1976-04-01,ANG MO KIO,NORTH-EAST REGION,28911.052240,38692.616791,1.366197,103.841505,Mayflower,800.632187,MRT,Brown,9142.409400,ANG MO KIO PRIMARY SCHOOL,403.611794
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9609,YISHUN,876,YISHUN ST 81,876,YISHUN STREET 81,NIL,760876,876 YISHUN STREET 81 SINGAPORE 760876,1987-12-01,YISHUN,NORTH REGION,28246.274546,44060.806242,1.414745,103.835532,Khatib,407.136706,MRT,Red,14551.599178,NAVAL BASE PRIMARY SCHOOL,402.052254
9610,YISHUN,877,YISHUN ST 81,877,YISHUN STREET 81,NIL,760877,877 YISHUN STREET 81 SINGAPORE 760877,1987-12-01,YISHUN,NORTH REGION,28237.634702,43967.636908,1.413902,103.835454,Khatib,473.260140,MRT,Red,14460.190304,NAVAL BASE PRIMARY SCHOOL,457.007677
9611,YISHUN,878,YISHUN ST 81,878,YISHUN STREET 81,NIL,760878,878 YISHUN STREET 81 SINGAPORE 760878,1988-01-01,YISHUN,NORTH REGION,28285.898091,43984.300584,1.414053,103.835888,Khatib,490.234213,MRT,Red,14470.867011,NAVAL BASE PRIMARY SCHOOL,408.316400
9612,YISHUN,879,YISHUN ST 81,879,YISHUN STREET 81,NIL,760879,879 YISHUN STREET 81 SINGAPORE 760879,1987-10-01,YISHUN,NORTH REGION,28311.512736,44027.290776,1.414442,103.836118,Khatib,477.228688,MRT,Red,14510.511874,NAVAL BASE PRIMARY SCHOOL,362.256294


#### 1a: First, check if there are any new addresses that appear in the new dataframe

In [22]:
df_raw.merge(
    masterlist_df,
    how="left",
    on=["town", "block", "street_name"],
    suffixes=["", "_r"],
    indicator=True,
).query("_merge != 'both'")

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,resale_price,floor_area_sqft,price_per_sqft,blk_no,road_name,building,postal,address,lease_commence_date,planning_area_ura,region_ura,x,y,latitude,longitude,closest_mrt_station,distance_to_mrt_meters,transport_type,line_color,distance_to_cbd,closest_pri_school,distance_to_pri_school_meters,_merge


#### 1b: If there are new addresses, use the following code to select the correct address, and insert ancillary information like floor area in square feet, price per square foot, and recalculating lease commence date

In [23]:
for_editing = masterlist_df[masterlist_df.isna().any(axis=1)]
for_editing = df_raw.merge(
    masterlist_df,
    how="left",
    on=["town", "block", "street_name"],
    suffixes=["", "_r"],
    indicator=True,
).query("_merge != 'both'").drop_duplicates(
    subset=["town", "block", "street_name"]
)

for_editing = for_editing.iloc[:, :11]

In [24]:
def obtain_lease_yearmth(row):
    today = pd.to_datetime(dt.date.today().replace(day=1))
    lease_commence = today - pd.DateOffset(
        years=row["lease_year"] + 99, months=row["lease_month"]
    )
    return lease_commence


for_editing["lease_year"] = for_editing["remaining_lease"].str.slice(0, 2).astype("int")
for_editing["lease_month"] = pd.to_numeric(
    for_editing["remaining_lease"].str.slice(9, 11), errors="coerce"
).fillna(0)


for_editing["lease_commence_date"] = for_editing.apply(obtain_lease_yearmth, axis=1)

for_editing = for_editing.drop(
    ["remaining_lease", "lease_year", "lease_month"], axis=1
)

KeyError: 'remaining_lease'

In [ ]:
if for_editing.shape[0] > 0:
    for i in for_editing.index:
        search_value = (
            for_editing.loc[i, "block"] + " " + for_editing.loc[i, "street_name"]
        )
        print(f"{i}: {search_value}")
        response = requests.get(
            f"https://www.onemap.gov.sg/api/common/elastic/search?searchVal={search_value}&returnGeom=Y&getAddrDetails=Y&pageNum=1"
        )
        while response.status_code != 200:
            time.sleep(1)
            print("not 200")
            response = requests.get(
                f"https://www.onemap.gov.sg/api/common/elastic/search?searchVal={search_value}&returnGeom=Y&getAddrDetails=Y&pageNum=1"
            )

        json_data = response.json()
        for j in json_data["results"]:
            print(j)
        time.sleep(0.5)
        user_input = input("Select the correct address: ")
        k = int(user_input) - 1
        print(k)

        if k == -2:
            for_editing.loc[i, "found"] = math.nan
            for_editing.loc[i, "search_val"] = math.nan
            for_editing.loc[i, "blk_no"] = math.nan
            for_editing.loc[i, "road_name"] = math.nan
            for_editing.loc[i, "building"] = math.nan
            for_editing.loc[i, "address"] = math.nan
            for_editing.loc[i, "postal"] = math.nan
            for_editing.loc[i, "x"] = math.nan
            for_editing.loc[i, "y"] = math.nan
            for_editing.loc[i, "latitude"] = math.nan
            for_editing.loc[i, "longitude"] = math.nan
        else:
            for_editing.loc[i, "found"] = 1
            for_editing.loc[i, "search_val"] = json_data["results"][k]["SEARCHVAL"]
            for_editing.loc[i, "blk_no"] = json_data["results"][k]["BLK_NO"]
            for_editing.loc[i, "road_name"] = json_data["results"][k]["ROAD_NAME"]
            for_editing.loc[i, "building"] = json_data["results"][k]["BUILDING"]
            for_editing.loc[i, "address"] = json_data["results"][k]["ADDRESS"]
            for_editing.loc[i, "postal"] = json_data["results"][k]["POSTAL"]
            for_editing.loc[i, "x"] = json_data["results"][k]["X"]
            for_editing.loc[i, "y"] = json_data["results"][k]["Y"]
            for_editing.loc[i, "latitude"] = json_data["results"][k]["LATITUDE"]
            for_editing.loc[i, "longitude"] = json_data["results"][k]["LONGITUDE"]

181263: 83A CIRCUIT RD
{'SEARCHVAL': 'MACPHERSON SPRING', 'BLK_NO': '83A', 'ROAD_NAME': 'CIRCUIT ROAD', 'BUILDING': 'MACPHERSON SPRING', 'ADDRESS': '83A CIRCUIT ROAD MACPHERSON SPRING SINGAPORE 371083', 'POSTAL': '371083', 'X': '34264.7055223584', 'Y': '33950.8719029184', 'LATITUDE': '1.32331350362685', 'LONGITUDE': '103.88961029001'}
0


In [ ]:
for_editing_gdf = gpd.GeoDataFrame(
    for_editing, geometry=gpd.points_from_xy(for_editing["x"], for_editing["y"])
)

for_editing_gdf

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,resale_price,floor_area_sqft,price_per_sqft,found,search_val,blk_no,road_name,building,address,postal,x,y,latitude,longitude,geometry
181263,2024-06-01,GEYLANG,4 ROOM,83A,CIRCUIT RD,16 TO 18,96.0,Model A,1830-06-01,985000.0,1033.3344,953.224822,1.0,MACPHERSON SPRING,83A,CIRCUIT ROAD,MACPHERSON SPRING,83A CIRCUIT ROAD MACPHERSON SPRING SINGAPORE 3...,371083,34264.7055223584,33950.8719029184,1.32331350362685,103.88961029001,POINT (34264.706 33950.872)


#### 1c. Appending MRT information

In [ ]:
mrt_file_loc = "../datasets/mrt_lrt_stations.csv"
mrt_gdf = gpd.read_file(mrt_file_loc, parse_dates=["OPENING"])
mrt_gdf["OPENING"] = pd.to_datetime(mrt_gdf["OPENING"])
# mrt_gdf.head()

In [ ]:
def find_closest_station(row, mrt_gdf):
    """Remove comments for the following 3 lines if you want the closest MRT at the time of transaction"""
    # mrt_gdf["OPENING_DATE"] = mrt_gdf["OPENING"].dt.to_period("M").dt.to_timestamp()
    # mrt_stations_filtered = mrt_gdf[mrt_gdf["OPENING"] < row["month"]]
    # distances = mrt_stations_filtered.distance(row["geometry"])

    """Remove comments for this line if you want the closest MRT station today"""
    distances = mrt_gdf.distance(row["geometry"])

    closest_station_index = distances.idxmin()
    shortest_distance = distances.min()

    closest_station_name = mrt_gdf.loc[closest_station_index, "STATION_NA"]
    closest_transport_type = mrt_gdf.loc[closest_station_index, "TYPE"]
    closest_mrt_color = mrt_gdf.loc[closest_station_index, "COLOR"]

    # distance to cbd
    raffles_place_index = mrt_gdf.query("STATION_NA == 'Raffles Place'").index[0]
    distance_to_cbd = mrt_gdf.loc[raffles_place_index, "geometry"].distance(
        row["geometry"]
    )

    return pd.Series(
        [
            closest_station_name,
            shortest_distance,
            closest_transport_type,
            closest_mrt_color,
            distance_to_cbd,
        ],
        index=[
            "closest_mrt_station",
            "distance_to_mrt_meters",
            "transport_type",
            "line_color",
            "distance_to_cbd",
        ],
    )


if for_editing_gdf.shape[0] > 0:
    for_editing_gdf[
        [
            "closest_mrt_station",
            "distance_to_mrt_meters",
            "transport_type",
            "line_color",
            "distance_to_cbd",
        ]
    ] = for_editing_gdf.apply(find_closest_station, mrt_gdf=mrt_gdf, axis=1)

In [ ]:
for_editing_gdf

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,resale_price,floor_area_sqft,price_per_sqft,found,search_val,blk_no,road_name,building,address,postal,x,y,latitude,longitude,geometry,closest_mrt_station,distance_to_mrt_meters,transport_type,line_color,distance_to_cbd
181263,2024-06-01,GEYLANG,4 ROOM,83A,CIRCUIT RD,16 TO 18,96.0,Model A,1830-06-01,985000.0,1033.3344,953.224822,1.0,MACPHERSON SPRING,83A,CIRCUIT ROAD,MACPHERSON SPRING,83A CIRCUIT ROAD MACPHERSON SPRING SINGAPORE 3...,371083,34264.7055223584,33950.8719029184,1.32331350362685,103.88961029001,POINT (34264.706 33950.872),MacPherson,315.584591,MRT,Blue,6066.433779


#### 1d. Appending Closest School information

In [ ]:
school_file_loc = "./schools_for_plotly.csv"
school_df = pd.read_csv(school_file_loc, index_col=0, dtype={"postal":"string"})
school_df["postal"] = school_df["postal"].astype("str").apply(lambda x: f"{x:0>6}")

# Convert the df into a gdf
school_gdf = gpd.GeoDataFrame(school_df, geometry=gpd.points_from_xy(school_df["x"], school_df["y"]))

In [ ]:
def find_closest_school(row, school_gdf, level="PRIMARY"):
    """Remove comments for the following 3 lines if you want the closest MRT at the time of transaction"""
    # school_gdf["OPENING_DATE"] = school_gdf["OPENING"].dt.to_period("M").dt.to_timestamp()
    # mrt_stations_filtered = school_gdf[school_gdf["OPENING"] < row["month"]]
    # distances = mrt_stations_filtered.distance(row["geometry"])

    school_gdf_filtered = school_gdf.query("mainlevel_code == @level")
    """Remove comments for this line if you want the closest MRT station today"""
    distances = school_gdf_filtered.distance(row["geometry"])

    closest_school_index = distances.idxmin()
    shortest_distance = distances.min()

    closest_school = school_gdf_filtered.loc[closest_school_index, "school_name"]

    return pd.Series(
        [
            closest_school,
            shortest_distance,
        ],
        index=[
            "closest_pri_school",
            "distance_to_pri_school_meters",
        ],
    )

for_editing_gdf[["closest_pri_school", "distance_to_pri_school_meters"]] = (
    for_editing_gdf.apply(find_closest_school, school_gdf=school_gdf, axis=1)
)

In [ ]:
for_editing_gdf

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,resale_price,floor_area_sqft,price_per_sqft,found,search_val,blk_no,road_name,building,address,postal,x,y,latitude,longitude,geometry,closest_mrt_station,distance_to_mrt_meters,transport_type,line_color,distance_to_cbd,closest_pri_school,distance_to_pri_school_meters
181263,2024-06-01,GEYLANG,4 ROOM,83A,CIRCUIT RD,16 TO 18,96.0,Model A,1830-06-01,985000.0,1033.3344,953.224822,1.0,MACPHERSON SPRING,83A,CIRCUIT ROAD,MACPHERSON SPRING,83A CIRCUIT ROAD MACPHERSON SPRING SINGAPORE 3...,371083,34264.7055223584,33950.8719029184,1.32331350362685,103.88961029001,POINT (34264.706 33950.872),MacPherson,315.584591,MRT,Blue,6066.433779,ST. MARGARET'S PRIMARY SCHOOL,896.659576


#### 1e. Appending URA planning area and regions

In [ ]:
geo_file_loc = "../datasets/sg_map/mp2014/MP14_PLNG_AREA_NO_SEA_PL.shp"
planning_areas_gdf = gpd.read_file(geo_file_loc)

planning_areas_gdf.head()

,OBJECTID,PLN_AREA_N,PLN_AREA_C,CA_IND,REGION_N,REGION_C,INC_CRC,FMEL_UPD_D,X_ADDR,Y_ADDR,SHAPE_Leng,SHAPE_Area,geometry
0,1,ANG MO KIO,AM,N,NORTH-EAST REGION,NER,E5CBDDE0C2113055,2016-05-11,28976.8763,40229.1238,17494.240190,1.394138e+07,"POLYGON ((30658.500 42047.527, 30679.195 42020..."
1,2,BEDOK,BD,N,EAST REGION,ER,1719251260799DF6,2016-05-11,38582.6650,34032.0961,21872.798962,2.173319e+07,"POLYGON ((38974.269 36138.243, 39371.471 35747..."
2,3,BISHAN,BS,N,CENTRAL REGION,CR,BA616285F402846F,2016-05-11,28789.7630,37450.8865,13517.121556,7.618921e+06,"POLYGON ((29772.191 38311.805, 29784.826 38304..."
3,4,BOON LAY,BL,N,WEST REGION,WR,A3DC87118B43CDED,2016-05-11,13410.3824,33008.9884,18528.467448,8.279408e+06,"POLYGON ((12861.383 32207.492, 12860.555 32208..."
4,5,BUKIT BATOK,BK,N,WEST REGION,WR,FB44C870B04B7F57,2016-05-11,19255.4150,37527.6527,15234.223423,1.113326e+07,"POLYGON ((20294.455 39114.528, 20334.318 39054..."


In [ ]:
for_editing_gdf = for_editing_gdf.sjoin(planning_areas_gdf[["PLN_AREA_N", "REGION_N", "geometry"]], how='left')

/Users/tim/miniforge3/envs/for_ml/lib/python3.10/site-packages/geopandas/geodataframe.py:2189: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: None
Right CRS: PROJCS["SVY21",GEOGCS["SVY21[WGS84]",DATUM["WGS_19 ...

  return geopandas.sjoin(left_df=self, right_df=df, *args, **kwargs)  # noqa: B026


In [ ]:
for_editing_gdf = for_editing_gdf.rename(columns={"PLN_AREA_N":"planning_area_ura", "REGION_N":"region_ura"})

In [ ]:
for_editing = for_editing_gdf[
    [
        "town",
        "block",
        "road_name",
        "blk_no",
        "street_name",
        "building",
        "postal",
        "address",
        "lease_commence_date",
        "planning_area_ura",
        "region_ura",
        "x",
        "y",
        "latitude",
        "longitude",
        "closest_mrt_station",
        "distance_to_mrt_meters",
        "transport_type",
        "line_color",
        "distance_to_cbd",
        "closest_pri_school",
        "distance_to_pri_school_meters",
    ]
]

In [ ]:
for_editing

,town,block,road_name,blk_no,street_name,building,postal,address,lease_commence_date,PLN_AREA_N,REGION_N,x,y,latitude,longitude,closest_mrt_station,distance_to_mrt_meters,transport_type,line_color,distance_to_cbd,closest_pri_school,distance_to_pri_school_meters
181263,GEYLANG,83A,CIRCUIT ROAD,83A,CIRCUIT RD,MACPHERSON SPRING,371083,83A CIRCUIT ROAD MACPHERSON SPRING SINGAPORE 3...,1830-06-01,GEYLANG,CENTRAL REGION,34264.7055223584,33950.8719029184,1.32331350362685,103.88961029001,MacPherson,315.584591,MRT,Blue,6066.433779,ST. MARGARET'S PRIMARY SCHOOL,896.659576


In [ ]:
new_masterlist_df = pd.concat([masterlist_df, for_editing], axis=0)
new_masterlist_df = new_masterlist_df.sort_values(by=["town", "street_name", "block"]).reset_index(drop=True)

In [ ]:
new_masterlist_df.head()

,town,block,street_name,blk_no,road_name,building,postal,address,lease_commence_date,planning_area_ura,region_ura,x,y,latitude,longitude,closest_mrt_station,distance_to_mrt_meters,transport_type,line_color,distance_to_cbd,closest_pri_school,distance_to_pri_school_meters,PLN_AREA_N,REGION_N
0,ANG MO KIO,205,ANG MO KIO AVE 1,205,ANG MO KIO AVENUE 1,NIL,560205,205 ANG MO KIO AVENUE 1 SINGAPORE 560205,1977-01-01,ANG MO KIO,NORTH-EAST REGION,29142.244275,38774.891527,1.366941,103.843582,Ang Mo Kio,742.768808,MRT,Red,9199.172507,ANG MO KIO PRIMARY SCHOOL,512.545254,NaN,NaN
1,ANG MO KIO,207,ANG MO KIO AVE 1,207,ANG MO KIO AVENUE 1,ANG MO KIO 22,560207,207 ANG MO KIO AVENUE 1 ANG MO KIO 22 SINGAPOR...,1976-07-01,ANG MO KIO,NORTH-EAST REGION,29060.485578,38651.052977,1.365821,103.842848,Ang Mo Kio,874.305666,MRT,Red,9084.169820,ANG MO KIO PRIMARY SCHOOL,527.275749,NaN,NaN
2,ANG MO KIO,208,ANG MO KIO AVE 1,208,ANG MO KIO AVENUE 1,ANG MO KIO 22,560208,208 ANG MO KIO AVENUE 1 ANG MO KIO 22 SINGAPOR...,1976-07-01,ANG MO KIO,NORTH-EAST REGION,29045.715075,38609.483079,1.365445,103.842715,Ang Mo Kio,908.966103,MRT,Red,9044.410980,ANG MO KIO PRIMARY SCHOOL,549.157654,NaN,NaN
3,ANG MO KIO,215,ANG MO KIO AVE 1,215,ANG MO KIO AVENUE 1,ANG MO KIO 22,560215,215 ANG MO KIO AVENUE 1 ANG MO KIO 22 SINGAPOR...,1976-04-01,ANG MO KIO,NORTH-EAST REGION,28924.303291,38732.591142,1.366558,103.841624,Mayflower,781.530965,MRT,Brown,9180.500545,ANG MO KIO PRIMARY SCHOOL,377.594832,NaN,NaN
4,ANG MO KIO,216,ANG MO KIO AVE 1,216,ANG MO KIO AVENUE 1,ANG MO KIO 22,560216,216 ANG MO KIO AVENUE 1 ANG MO KIO 22 SINGAPOR...,1976-04-01,ANG MO KIO,NORTH-EAST REGION,28911.05224,38692.616791,1.366197,103.841505,Mayflower,800.632187,MRT,Brown,9142.409400,ANG MO KIO PRIMARY SCHOOL,403.611794,NaN,NaN


In [ ]:
new_masterlist_df.to_csv(masterlist_file_loc)

### 2. Append additional information to the raw dataset

#### 2a: Reopen Masterlist

In [ ]:
masterlist_file_loc = "../datasets/hdb_resale_flat_address_masterlist.csv"
# masterlist_file_loc = "./temp_masterlist.csv"
masterlist_df = pd.read_csv(
    masterlist_file_loc,
    index_col=0,
    dtype={"postal": "object"},
    parse_dates=["lease_commence_date"],
)

masterlist_df.head(1)

,town,block,street_name,blk_no,road_name,building,postal,address,lease_commence_date,planning_area_ura,region_ura,x,y,latitude,longitude,closest_mrt_station,distance_to_mrt_meters,transport_type,line_color,distance_to_cbd,closest_pri_school,distance_to_pri_school_meters
0,ANG MO KIO,205,ANG MO KIO AVE 1,205,ANG MO KIO AVENUE 1,NIL,560205,205 ANG MO KIO AVENUE 1 SINGAPORE 560205,1977-01-01,ANG MO KIO,NORTH-EAST REGION,29142.244275,38774.891527,1.366941,103.843582,Ang Mo Kio,742.768808,MRT,Red,9199.172507,ANG MO KIO PRIMARY SCHOOL,512.545254


#### 2b: Perform a left merge between the new HDB dataset and the address masterlist

In [ ]:
df_new = df_raw.merge(
    masterlist_df,
    how="left",
    on=["town", "block", "street_name"],
    suffixes=["", "_r"],
    indicator=True,
)

In [ ]:
df_new.query("_merge != 'both'")

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,resale_price,floor_area_sqft,price_per_sqft,blk_no,road_name,building,postal,address,lease_commence_date,planning_area_ura,region_ura,x,y,latitude,longitude,closest_mrt_station,distance_to_mrt_meters,transport_type,line_color,distance_to_cbd,closest_pri_school,distance_to_pri_school_meters,_merge


In [ ]:
df_new = df_new.drop("_merge", axis=1)

In [ ]:
file_destination = file_loc_raw.replace("raw", "coords_mrt")
file_destination
df_new.to_csv(file_destination)

#### The following is used to call the Onemap API and populate coordinate details (deprecated, use the above masterlist instead)

In [ ]:
# # Obtain geospatial coordinates with the Onemap API

# def get_coordinates_for_each_row(row):
#     search_value = row["block"] + " " + row["street_name"]
#     # print(search_value)

#     response = requests.get(f"https://www.onemap.gov.sg/api/common/elastic/search?searchVal={search_value}&returnGeom=Y&getAddrDetails=Y&pageNum=1")
#     while response.status_code != 200:
#         time.sleep(1)
#         print("not 200")
#         response = requests.get(f"https://www.onemap.gov.sg/api/common/elastic/search?searchVal={search_value}&returnGeom=Y&getAddrDetails=Y&pageNum=1")

#     data = response.json()

#     row_data = data['results'][0]
#     searchval = row_data["SEARCHVAL"]
#     address = row_data["ADDRESS"]
#     postal = row_data["POSTAL"]
#     x = row_data["X"]
#     y = row_data["Y"]
#     latitude = row_data["LATITUDE"]
#     longitude = row_data["LONGITUDE"]

#     return pd.Series([address, postal, x, y, latitude, longitude], 
#                      index=["SEARCHVAL", "ADDRESS", "POSTAL", "X", "Y", "LATITUDE", "LONGITUDE"])

In [ ]:
# Appending additional columns to the dataframe


# NOTE: SHOULDNT APPEND
# for_addition["year"] = for_addition["month"].dt.year
# for_addition["lease_years"] = for_addition["remaining_lease"].str.split(" ").apply(lambda x: int(x[0]))
# bins = pd.IntervalIndex.from_tuples(
#     [(40, 50), (50, 60), (60, 70), (70, 80), (80, 90), (90, 100)]
# )
# for_addition["lease_cat"] = pd.cut(for_addition["lease_years"], bins)
# for_addition = for_addition.drop("index", axis=1)

#### Checking datasets

In [ ]:
file_loc_raw = "../datasets/resale_hdb_price_coords_mrt_13jun.csv"
df_jun13 = pd.read_csv(
    file_loc_raw,
    parse_dates=["month", "lease_commence_date"],
    index_col=0,
    dtype={"x": "float64", "y": "float64", "postal": "object"},
    low_memory=False,
)

In [ ]:
df_jun13

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,resale_price,floor_area_sqft,price_per_sqft,blk_no,road_name,building,postal,address,lease_commence_date,planning_area_ura,region_ura,x,y,latitude,longitude,closest_mrt_station,distance_to_mrt_meters,transport_type,line_color,distance_to_cbd,closest_pri_school,distance_to_pri_school_meters,_merge
0,2017-01-01,ANG MO KIO,2 ROOM,406,ANG MO KIO AVE 10,10 TO 12,44.0,Improved,232000.0,473.6116,489.852867,406,ANG MO KIO AVENUE 10,NIL,560406,406 ANG MO KIO AVENUE 10 SINGAPORE 560406,1979-05-01,ANG MO KIO,NORTH-EAST REGION,30288.234663,38229.067463,1.362005,103.853880,Ang Mo Kio,999.941618,MRT,Red,8615.656983,TOWNSVILLE PRIMARY SCHOOL,218.125254,both
1,2017-01-01,ANG MO KIO,3 ROOM,108,ANG MO KIO AVE 4,01 TO 03,67.0,New Generation,250000.0,721.1813,346.653470,108,ANG MO KIO AVENUE 4,KEBUN BARU HEIGHTS,560108,108 ANG MO KIO AVENUE 4 KEBUN BARU HEIGHTS SIN...,1978-08-01,ANG MO KIO,NORTH-EAST REGION,28543.458747,39220.009892,1.370966,103.838202,Mayflower,189.980291,MRT,Brown,9715.131951,ANG MO KIO PRIMARY SCHOOL,241.572335,both
2,2017-01-01,ANG MO KIO,3 ROOM,602,ANG MO KIO AVE 5,01 TO 03,67.0,New Generation,262000.0,721.1813,363.292836,602,ANG MO KIO AVENUE 5,YIO CHU KANG GREEN,560602,602 ANG MO KIO AVENUE 5 YIO CHU KANG GREEN SIN...,1980-06-01,ANG MO KIO,NORTH-EAST REGION,28228.099954,40297.283149,1.380709,103.835368,Lentor,532.154773,MRT,Brown,10828.819556,ANDERSON PRIMARY SCHOOL,777.155378,both
3,2017-01-01,ANG MO KIO,3 ROOM,465,ANG MO KIO AVE 10,04 TO 06,68.0,New Generation,265000.0,731.9452,362.048962,465,ANG MO KIO AVENUE 10,TECK GHEE HORIZON,560465,465 ANG MO KIO AVENUE 10 TECK GHEE HORIZON SIN...,1980-02-01,ANG MO KIO,NORTH-EAST REGION,30657.824693,38693.098657,1.366201,103.857201,Ang Mo Kio,945.371842,MRT,Red,9097.929095,TECK GHEE PRIMARY SCHOOL,698.165530,both
4,2017-01-01,ANG MO KIO,3 ROOM,601,ANG MO KIO AVE 5,01 TO 03,67.0,New Generation,265000.0,721.1813,367.452678,601,ANG MO KIO AVENUE 5,YIO CHU KANG GREEN,560601,601 ANG MO KIO AVENUE 5 YIO CHU KANG GREEN SIN...,1980-06-01,ANG MO KIO,NORTH-EAST REGION,28201.782245,40334.052030,1.381041,103.835132,Lentor,498.418205,MRT,Brown,10869.453109,ANDERSON PRIMARY SCHOOL,782.553222,both
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
181561,2024-05-01,YISHUN,EXECUTIVE,834,YISHUN ST 81,04 TO 06,154.0,Maisonette,948000.0,1657.6406,571.897189,834,YISHUN STREET 81,NIL,760834,834 YISHUN STREET 81 SINGAPORE 760834,1988-01-01,YISHUN,NORTH REGION,28075.777355,44098.765290,1.415088,103.834000,Khatib,278.011616,MRT,Red,14610.975312,PEIYING PRIMARY SCHOOL,463.909479,both
181562,2024-06-01,YISHUN,EXECUTIVE,826,YISHUN ST 81,10 TO 12,146.0,Maisonette,1000000.0,1571.5294,636.322808,826,YISHUN STREET 81,NIL,760826,826 YISHUN STREET 81 SINGAPORE 760826,1988-01-01,YISHUN,NORTH REGION,27903.805992,43997.703489,1.414174,103.832454,Khatib,359.654884,MRT,Red,14534.848600,PEIYING PRIMARY SCHOOL,407.218879,both
181563,2024-06-01,YISHUN,EXECUTIVE,826,YISHUN ST 81,04 TO 06,146.0,Maisonette,1000000.0,1571.5294,636.322808,826,YISHUN STREET 81,NIL,760826,826 YISHUN STREET 81 SINGAPORE 760826,1988-01-01,YISHUN,NORTH REGION,27903.805992,43997.703489,1.414174,103.832454,Khatib,359.654884,MRT,Red,14534.848600,PEIYING PRIMARY SCHOOL,407.218879,both
181564,2024-02-01,YISHUN,MULTI-GENERATION,666,YISHUN AVE 4,04 TO 06,164.0,Multi Generation,998000.0,1765.2796,565.349534,666,YISHUN AVENUE 4,NIL,760666,666 YISHUN AVENUE 4 SINGAPORE 760666,1987-12-01,YISHUN,NORTH REGION,28806.756365,44531.159086,1.418998,103.840568,Khatib,863.123975,MRT,Red,14962.748464,NORTHLAND PRIMARY SCHOOL,221.649812,both
